# 1. Importing Required Libraries

In [1]:
import numpy as np
import pandas as pd

# 2. Loading Datasets

In [2]:
# Load the application_record and credit_record datasets
application_df = pd.read_csv('data/application_record.csv')
credit_df = pd.read_csv('data/credit_record.csv')